In [1]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
# glove embedding
!kaggle competitions download -c quora-insincere-questions-classification

 75% 41.0M/54.9M [00:00<00:00, 54.9MB/s]
100% 54.9M/54.9M [00:00<00:00, 139MB/s] 
100% 4.09M/4.09M [00:00<00:00, 41.9MB/s]

100% 5.94G/5.96G [00:49<00:00, 163MB/s]
100% 5.96G/5.96G [00:49<00:00, 130MB/s]
 32% 5.00M/15.8M [00:00<00:00, 24.7MB/s]
100% 15.8M/15.8M [00:00<00:00, 60.3MB/s]


In [3]:
!unzip /content/embeddings.zip > /dev/null

# Exploring and Cleaning the data

In [4]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_colwidth', None)

import warnings
warnings.filterwarnings("ignore")

In [5]:
data = pd.read_csv('train_aug_in_kag_org.csv')

In [6]:
data.head(10)

,title,text,title_text,label
0,squirrel financial wellbeing,A Squirrel account makes it easy to stick to your budget and save. Start for free today!,squirrel financial wellbeing. A Squirrel account makes it easy to stick to your budget and save. Start for free today!,payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
1,senior insurance marketing,"Welcome to the home page of Senior Insurance Marketing, a full service brokerage specializing in Medicare and senior insurance products. Located in Lincoln, NE.","senior insurance marketing. Welcome to the home page of Senior Insurance Marketing, a full service brokerage specializing in Medicare and senior insurance products. Located in Lincoln, NE.",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
2,3i financial group inc.,"A Small Boutique Firm Dedicated to Achieving Our Clients Goals.#sep#inform.invest.insure Get a Free, No Obligation Quote forAllYour Insurance Needs. 3i Financial offers a wide variety of insurance products to help suit your needs. Choose from individual, family, and group. GET A QUOTE TODAY Travel Insurance Get peace of mind with comprehensive travel insurance. Get a free quote today! Health & Dental Get affordable health Read more","3i financial group inc.. A Small Boutique Firm Dedicated to Achieving Our Clients Goals.#sep#inform.invest.insure Get a Free, No Obligation Quote forAllYour Insurance Needs. 3i Financial offers a wide variety of insurance products to help suit your needs. Choose from individual, family, and group. GET A QUOTE TODAY Travel Insurance Get peace of mind with comprehensive travel insurance. Get a free quote today! Health & Dental Get affordable health Read more",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
3,brownstone insurance,"Brownstone Insurance provides master insurance policies for multi-unit buildings throughout Massachusetts, as well as personal insurance products such as renter's insurance and homeowner's insurance.#sep#Brownstone Insurance provides master insurance policies for multi-unit \nbuildings throughout Massachusetts, as well as personal insurance products \nsuch as renter's insurance and homeowner's insurance.","brownstone insurance. Brownstone Insurance provides master insurance policies for multi-unit buildings throughout Massachusetts, as well as personal insurance products such as renter's insurance and homeowner's insurance.#sep#Brownstone Insurance provides master insurance policies for multi-unit \nbuildings throughout Massachusetts, as well as personal insurance products \nsuch as renter's insurance and homeowner's insurance.",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
4,pacificwide lending,Learn more about Pacificwide Lending and our commercial & residential loan services in California.,pacificwide lending. Learn more about Pacificwide Lending and our commercial & residential loan services in California.,payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
5,city auto finance,"City Auto Finance is the Southeast's most trusted used-car floor plan financing company with locations in Memphis, TN; Mobile, AL; Huntsville, AL and Murfreesboro, TN. We provide you with competitive rates and the capability to make payments online.","city auto finance. City Auto Finance is the Southeast's most trusted used-car floor plan financing company with locations in Memphis, TN; Mobile, AL; Huntsville, AL and Murfreesboro, TN. We provide you with competitive rates and the capability to make payments online.",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
6,"veterans financial, inc.","VA Aid and Attendance benefit can help pay for assisted living, home care or nursing home. Veterans Financial Inc can advise you at no cost about benefits","veterans financial, inc.. VA Aid and Attendance benefit can help pay for assisted living, home care or nursing home. Veterans Financial Inc can advise you at no cost 

In [7]:
data.shape

(34953, 4)

In [8]:
data.isnull().sum()

title         42
text           0
title_text     0
label          0
dtype: int64

In [9]:
data.dropna(inplace=True)

In [10]:
data.drop(columns=['title', 'text'], inplace=True)

In [11]:
data.head()

,title_text,label
0,squirrel financial wellbeing. A Squirrel account makes it easy to stick to your budget and save. Start for free today!,payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
1,"senior insurance marketing. Welcome to the home page of Senior Insurance Marketing, a full service brokerage specializing in Medicare and senior insurance products. Located in Lincoln, NE.",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
2,"3i financial group inc.. A Small Boutique Firm Dedicated to Achieving Our Clients Goals.#sep#inform.invest.insure Get a Free, No Obligation Quote forAllYour Insurance Needs. 3i Financial offers a wide variety of insurance products to help suit your needs. Choose from individual, family, and group. GET A QUOTE TODAY Travel Insurance Get peace of mind with comprehensive travel insurance. Get a free quote today! Health & Dental Get affordable health Read more",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
3,"brownstone insurance. Brownstone Insurance provides master insurance policies for multi-unit buildings throughout Massachusetts, as well as personal insurance products such as renter's insurance and homeowner's insurance.#sep#Brownstone Insurance provides master insurance policies for multi-unit \nbuildings throughout Massachusetts, as well as personal insurance products \nsuch as renter's insurance and homeowner's insurance.",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
4,pacificwide lending. Learn more about Pacificwide Lending and our commercial & residential loan services in California.,payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing


# Preparing our data for modeling 

In [12]:
import re
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
# text
text_corpus = []
lm = WordNetLemmatizer()
words = stopwords.words('english')

for i in range(data.shape[0]):
    row = re.sub('[^a-zA-Z\s]', '', data.iloc[i, 0]).lower()
    row = row.split()  
    row = [lm.lemmatize(word) for word in row if word not in words]
    row = ' '.join(row)
    text_corpus.append(row)

In [14]:
len(text_corpus)

34911

In [15]:
text_corpus[0]

'squirrel financial wellbeing squirrel account make easy stick budget save start free today'

_Our Bag of Words_

In [16]:
X = np.array(text_corpus)
y = data.iloc[:, 1].values

In [17]:
X.shape, y.shape

((34911,), (34911,))

In [18]:
# LabelEncoder
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)

In [19]:
np.unique(y)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

# Preparing for modeling 

In [20]:
from tqdm import tqdm

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [21]:
token = Tokenizer()

In [22]:
token.fit_on_texts(X)
seq = token.texts_to_sequences(X)

In [23]:
len(token.word_index)

70178

In [24]:
vocab_size = len(token.word_index)+1    #OOV token

In [25]:
l_seq = map(lambda x: len(x), seq)
max(l_seq)

1766

In [26]:
pad_seq = pad_sequences(seq, maxlen=1000)

In [27]:
# Splitting the data
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(pad_seq, y, test_size = 0.2, random_state = 42)

In [28]:
X_train.shape, X_val.shape

((27928, 1000), (6983, 1000))

In [29]:
embedding_vector = {}
with open('glove.840B.300d/glove.840B.300d.txt') as f:
    for line in tqdm(f):
        value = line.split(' ')
        word = value[0]
        coef = np.array(value[1:],dtype = 'float32')
        embedding_vector[word] = coef

2196017it [02:08, 17123.87it/s]


In [30]:
embedding_matrix = np.zeros((vocab_size, 300))
for word, idx in tqdm(token.word_index.items()):
    embedding_value = embedding_vector.get(word)
    if embedding_value is not None:
        embedding_matrix[idx] = embedding_value

100%|██████████| 70178/70178 [00:00<00:00, 513411.17it/s]


# Modeling

In [31]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, CuDNNLSTM, Bidirectional
from keras.callbacks import EarlyStopping #ModelCheckpoint, Callback, 

In [33]:
model = Sequential()
model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=1000, trainable = False))
model.add(LSTM(128))
model.add(Dense(64,activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(32,activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(15,activation = 'softmax'))

In [34]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [35]:
early = EarlyStopping(monitor="val_accuracy", mode='max', patience=5, restore_best_weights=True)

In [36]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs = 50, batch_size=256, callbacks=[early]) 

Epoch 1/50
110/110 [==============================] - 23s 154ms/step - loss: 1.3820 - accuracy: 0.6016 - val_loss: 0.8063 - val_accuracy: 0.7873
Epoch 2/50
110/110 [==============================] - 17s 150ms/step - loss: 0.7976 - accuracy: 0.7962 - val_loss: 0.6666 - val_accuracy: 0.8213
Epoch 3/50
110/110 [==============================] - 17s 151ms/step - loss: 0.6761 - accuracy: 0.8224 - val_loss: 0.5997 - val_accuracy: 0.8383
Epoch 4/50
110/110 [==============================] - 17s 152ms/step - loss: 0.6005 - accuracy: 0.8395 - val_loss: 0.5475 - val_accuracy: 0.8465
Epoch 5/50
110/110 [==============================] - 17s 154ms/step - loss: 0.5259 - accuracy: 0.8576 - val_loss: 0.4991 - val_accuracy: 0.8617
Epoch 6/50
110/110 [==============================] - 17s 155ms/step - loss: 0.4695 - accuracy: 0.8709 - val_loss: 0.4947 - val_accuracy: 0.8621
Epoch 7/50
110/110 [==============================] - 17s 155ms/step - loss: 0.4236 - accuracy: 0.8836 - val_loss: 0.4631 - val_ac

In [37]:
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score

columns = ['Accuracy', 'Precision-MacroAVG', 'Precision-WeightedAVG', 'Recall-MacroAVG', 'Recall-WeightedAVG', 
           'F1_score-MacroAVG','F1_score-WeightedAVG']

y_pred = model.predict(X_val)
y_pred = np.argmax(y_pred, axis=1)

record = {}
record['Accuracy'] = accuracy_score(y_val, y_pred)*100
record['Precision-MacroAVG'] = precision_score(y_val, y_pred, average='macro')*100
record['Precision-WeightedAVG'] = precision_score(y_val, y_pred, average='weighted')*100
record['Recall-MacroAVG'] = recall_score(y_val, y_pred, average='macro')*100
record['Recall-WeightedAVG'] = recall_score(y_val, y_pred, average='weighted')*100
record['F1_score-MacroAVG'] = f1_score(y_val, y_pred, average='macro')*100
record['F1_score-WeightedAVG'] = f1_score(y_val, y_pred, average='weighted')*100

results = pd.Series(record, index=columns)

In [38]:
results

Accuracy                 91.264499
Precision-MacroAVG       84.868629
Precision-WeightedAVG    91.242120
Recall-MacroAVG          81.948529
Recall-WeightedAVG       91.264499
F1_score-MacroAVG        82.341465
F1_score-WeightedAVG     91.185461
dtype: float64

In [39]:
model.save('DeepApproach.h5')

# Testing on test_data

In [40]:
test_data = pd.read_csv('test_in_kag_org.csv')

In [41]:
test_data.shape

(6501, 4)

In [42]:
test_data.isnull().sum()

title         0
text          0
title_text    0
label         0
dtype: int64

In [43]:
test_data.drop(columns=['title', 'text'], inplace=True)

# Preparing our data for modeling 

In [44]:
# text
text_corpus_test = []
lm = WordNetLemmatizer()
words = stopwords.words('english')

for i in range(test_data.shape[0]):
    row = re.sub('[^a-zA-Z\s]', '', test_data.iloc[i, 0]).lower()
    row = row.split()  
    row = [lm.lemmatize(word) for word in row if word not in words]
    row = ' '.join(row)
    text_corpus_test.append(row)

In [45]:
len(text_corpus_test)

6501

In [46]:
text_corpus_test[0]

'centralized hub procore offer ability communicate team get full view project one place assign task stay connected google map google doc'

_Our Bag of Words_

In [47]:
X_text_test = token.texts_to_sequences(text_corpus_test)
pad_seq_test = pad_sequences(X_text_test, maxlen=1000)

In [48]:
X_test = pad_seq_test.copy()
y_test = test_data.iloc[:, 1].values

In [49]:
X_test.shape, y_test.shape

((6501, 1000), (6501,))

In [50]:
y_test = le.transform(y_test)

In [51]:
np.unique(y_test)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [52]:
tf.keras.models.load_model('DeepApproach.h5')

In [53]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

record = {}
record['Accuracy'] = accuracy_score(y_test, y_pred)*100
record['Precision-MacroAVG'] = precision_score(y_test, y_pred, average='macro')*100
record['Precision-WeightedAVG'] = precision_score(y_test, y_pred, average='weighted')*100
record['Recall-MacroAVG'] = recall_score(y_test, y_pred, average='macro')*100
record['Recall-WeightedAVG'] = recall_score(y_test, y_pred, average='weighted')*100
record['F1_score-MacroAVG'] = f1_score(y_test, y_pred, average='macro')*100
record['F1_score-WeightedAVG'] = f1_score(y_test, y_pred, average='weighted')*100

results = pd.Series(record, index=columns)
results

Accuracy                 91.124442
Precision-MacroAVG       70.267597
Precision-WeightedAVG    90.944077
Recall-MacroAVG          69.371517
Recall-WeightedAVG       91.124442
F1_score-MacroAVG        69.212405
F1_score-WeightedAVG     90.956851
dtype: float64